In [46]:
import numpy as np
import statistics

In [47]:
class KNN_Classifier():
    
    # initializing parameters
    def __init__(self, distance_type):
        
        self.distance_type = distance_type
        
    # getting distance
    def get_distance(self, training_data_point, test_data_point):
        
        if (self.distance_type == 'Euclidean'):
            
            dist = 0
            
            for i in range(len(training_data_point)-1):  # As the last value will be our class label we are subtracting one
                
                dist = dist + (training_data_point[i]-test_data_point[i])**2    # sqrt((x1-x2)^2 + (y1-y2)^2)
                
            euclidean_distance = np.sqrt(dist)
            
            return euclidean_distance
        
        elif (self.distance_type == 'Manhatten'):
            
            dist = 0
            
            for i in range(len(training_data_point)-1): # As the last value will be our class label we are subtracting one
                
                dist = dist + abs(training_data_point[i]-test_data_point[i])   # |x1-x2| + |y1-y2|
                
            manhatten_distance = dist
            
            return manhatten_distance
        
    # getting nearest neighbors    
    def nearest_neighbors(self, X_train, X_test, k):
        
        distance_list = []
        
        for training_data in X_train:
            
            distance = self.get_distance(training_data, X_test)
            distance_list.append((training_data, distance))
            
        distance_list.sort(key = lambda x:x[1])  # sorting distance list based on distance
        
        neighbors_list = []
        
        for i in range(k):
            
            neighbors_list.append(distance_list[i][0])  # extracting the nearest data point from the distance list
            
        return neighbors_list
            
    # predicting class for test data  
    def predict(self, X_train, X_test, k):
        
        neighbors = self.nearest_neighbors(X_train, X_test, k)
        label_list = []
        
        for data in neighbors:
            label_list.append(data[-1])  # Extracting class label values of the nearest neighbors
            
        predicted_label = statistics.mode(label_list)  # taking the common one 
        
        return predicted_label
    
        
        
        
        

In [48]:
# importing dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Import dataset and perdorming basic data analysis

In [49]:
diabetes_dataset = pd.read_csv("diabetes.csv")

In [50]:
# printing the first 5 rows
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [51]:
# printing last five rows
diabetes_dataset.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [52]:
diabetes_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [53]:
# printing statistics for our data
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [54]:
diabetes_dataset['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

# Splitting the data into features and target

In [55]:
X = diabetes_dataset.drop( columns = 'Outcome', axis = 1)
Y = diabetes_dataset['Outcome']

In [56]:
print(X)
print(Y)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [57]:
# Converting our features and target to numpy array type to perform functions easily
X = X.to_numpy()
Y = Y.to_numpy()

# Splitting features and target for training and testing

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 2)

In [59]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


In [60]:
print(X_train)

[[1.000e+00 1.240e+02 7.400e+01 ... 2.780e+01 1.000e-01 3.000e+01]
 [1.000e+00 9.500e+01 8.200e+01 ... 3.500e+01 2.330e-01 4.300e+01]
 [0.000e+00 9.400e+01 7.000e+01 ... 4.350e+01 3.470e-01 2.100e+01]
 ...
 [4.000e+00 1.250e+02 7.000e+01 ... 2.890e+01 1.144e+00 4.500e+01]
 [3.000e+00 1.160e+02 7.400e+01 ... 2.630e+01 1.070e-01 2.400e+01]
 [4.000e+00 1.100e+02 6.600e+01 ... 3.190e+01 4.710e-01 2.900e+01]]


In [61]:
# Adding Y_train to X_train as we will need the class labels to predict values in X_test
X_train = np.insert(X_train, 8 , Y_train, axis = 1)
print(X_train)

[[1.000e+00 1.240e+02 7.400e+01 ... 1.000e-01 3.000e+01 0.000e+00]
 [1.000e+00 9.500e+01 8.200e+01 ... 2.330e-01 4.300e+01 1.000e+00]
 [0.000e+00 9.400e+01 7.000e+01 ... 3.470e-01 2.100e+01 0.000e+00]
 ...
 [4.000e+00 1.250e+02 7.000e+01 ... 1.144e+00 4.500e+01 1.000e+00]
 [3.000e+00 1.160e+02 7.400e+01 ... 1.070e-01 2.400e+01 0.000e+00]
 [4.000e+00 1.100e+02 6.600e+01 ... 4.710e-01 2.900e+01 0.000e+00]]


# Model predictions

In [62]:
model = KNN_Classifier(distance_type="Euclidean")

In [63]:
# model can only predict for one value
model.predict(X_train, X_test[0], k = 5)

0.0

In [64]:
print(Y_test[0])

0


# Creating a loop to get all the predicted values at once

In [65]:
Predicted_list_labels = []

for x in range(len(X_test)):
    prediction  = model.predict(X_train, X_test[x], k = 2)
    Predicted_list_labels.append(prediction)
    
print(Predicted_list_labels)

[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# Checking model accuracy

In [66]:
accuracy = accuracy_score(Y_test, Predicted_list_labels)
print(f"The accuracy for our KNN model is : {round(accuracy*100,2)}%")

The accuracy for our KNN model is : 69.48%
